# Roommate Matching Data Preprocessing

This notebook preprocesses the U-M Ann Arbor Roommate Question Data for use in a Deep Clustering Network.

In [8]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
!pip install openpyxl

# For reproducibility
np.random.seed(42)


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## 1. Load and Examine Data

In [9]:
# Load the raw data
df = pd.read_excel('U-M Ann Arbor Roommate Question Data.xlsx')

# Basic info about the dataset
print('Dataset shape:', df.shape)
print('\nColumns:', df.columns.tolist())
print('\nSample of data:')
display(df.head())

Dataset shape: (174479, 13)

Columns: ['Anonymized Student Number', 'Question Type ID', 'Question Type Description', 'Question Category ID', 'Question Category Description', 'Student Answer Weight ID', 'Student Answer Weight Description', 'Question Text', 'Low Answer Weight Description', 'Medium Low Answer Weight Description', 'Medium Answer Weight Description', 'Medium High Answer Weight Description', 'High Answer Weight Description']

Sample of data:


,Anonymized Student Number,Question Type ID,Question Type Description,Question Category ID,Question Category Description,Student Answer Weight ID,Student Answer Weight Description,Question Text,Low Answer Weight Description,Medium Low Answer Weight Description,Medium Answer Weight Description,Medium High Answer Weight Description,High Answer Weight Description
0,1000,1,Roommate Matching Questions,1,Laundry,1,Low,How often do you plan on doing your laundry?,Weekly,NaN,Bi-weekly,NaN,Monthly
1,1000,1,Roommate Matching Questions,4,Weekend Activities,3,Medium,How do you see yourself usually spending Frida...,Leaving the campus,Socializing with friends in the residence hall,Socializing with friends outside of the reside...,Spending time alone,Studying
2,1000,1,Roommate Matching Questions,5,Sharing Belongings,1,Low,How comfortable are you sharing belongings wit...,My items are their items,NaN,I am open to sharing some items,NaN,I do not want to share any items with my roomm...
3,1000,1,Roommate Matching Questions,21,Studying,3,Medium,What kind of studying should happen in your room?,Studying in the room all the time is ideal,NaN,"I prefer a mix, I like to study everywhere inc...",NaN,I prefer all studying to be out of the room
4,1000,1,Roommate Matching Questions,22,Room Condition,3,Medium,Which statement best describes your standards ...,I like my room to be clean most of the time.,NaN,I like my room to be generally neat but can ha...,NaN,It does not bother me if my room is cluttered ...


In [10]:
# Check unique values and missing data
print('Unique students:', df['Anonymized Student Number'].nunique())
print('Unique questions:', df['Question Text'].nunique())
print('\nMissing values per column:')
print(df.isnull().sum())

Unique students: 14697
Unique questions: 12

Missing values per column:
Anonymized Student Number                     0
Question Type ID                              0
Question Type Description                     0
Question Category ID                          0
Question Category Description                 0
Student Answer Weight ID                      0
Student Answer Weight Description         20636
Question Text                                 0
Low Answer Weight Description                 0
Medium Low Answer Weight Description     145375
Medium Answer Weight Description              0
Medium High Answer Weight Description    145375
High Answer Weight Description                0
dtype: int64


## 2. Understanding Questions and Responses

In [11]:
# Look at all unique questions
questions = df['Question Text'].unique()
print('All questions in survey:')
for i, q in enumerate(questions, 1):
    print(f'{i}. {q}')

All questions in survey:
1. How often do you plan on doing your laundry?
2. How do you see yourself usually spending Friday evenings?
3. How comfortable are you sharing belongings with a roommate? Examples may include: fridge, food, cooking items, clothes, electronics
4. What kind of studying should happen in your room?
5. Which statement best describes your standards for the condition of your room/apartment?
6. How clean do you want your room to be?
7. How comfortable are you with any individual in the room bringing overnight guests?
8. How do you feel about groups of people socializing in your room/apartment?
9. How would you describe your attitude toward smoking or vaping?
10. How would you describe your attitude toward the use of alcohol in on-campus living?
11. Dealing with conflicts:
12. How comfortable are you with adjusting your living habits to meet a roommateâ€™s dietary restrictions or allergies (e.g., milk, eggs, fish, shellfish, tree nuts, peanuts, wheat, soy)?


In [12]:
# Check the distribution of answers for each question
for question in questions:
    question_data = df[df['Question Text'] == question]
    print(f'\nQuestion: {question}')
    print('Response distribution:')
    print(question_data['Student Answer Weight ID'].value_counts().sort_index())


Question: How often do you plan on doing your laundry?
Response distribution:
Student Answer Weight ID
1    11393
3     3132
5       53
Name: count, dtype: int64

Question: How do you see yourself usually spending Friday evenings?
Response distribution:
Student Answer Weight ID
1     605
2    3058
3    8966
4     986
5     896
Name: count, dtype: int64

Question: How comfortable are you sharing belongings with a roommate? Examples may include: fridge, food, cooking items, clothes, electronics
Response distribution:
Student Answer Weight ID
1    9422
3    4498
5     676
Name: count, dtype: int64

Question: What kind of studying should happen in your room?
Response distribution:
Student Answer Weight ID
1    3007
2     316
3    4981
4    5248
5    1069
Name: count, dtype: int64

Question: Which statement best describes your standards for the condition of your room/apartment?
Response distribution:
Student Answer Weight ID
1    5289
3    8857
5     470
Name: count, dtype: int64

Question

## 3. Reshape Data (Long to Wide Format)

In [13]:
# Pivot the data to get one row per student
wide_df = df.pivot(
    index='Anonymized Student Number',
    columns='Question Text',
    values='Student Answer Weight ID'
)

# Reset index to make student number a regular column
wide_df = wide_df.reset_index()

# Clean up column names (remove spaces and special characters)
wide_df.columns = [col if col == 'Anonymized Student Number' 
                  else f'Q{i+1}' 
                  for i, col in enumerate(wide_df.columns)]

print('Shape after reshaping:', wide_df.shape)
print('\nFirst few rows:')
display(wide_df.head())

Shape after reshaping: (14697, 13)

First few rows:


,Anonymized Student Number,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13
0,1000,5.0,1.0,1.0,5.0,3.0,3.0,3.0,1.0,1.0,1.0,3.0,3.0
1,1001,1.0,2.0,3.0,5.0,5.0,5.0,3.0,1.0,1.0,1.0,1.0,3.0
2,1002,1.0,2.0,3.0,5.0,5.0,3.0,3.0,1.0,1.0,3.0,1.0,1.0
3,1003,1.0,2.0,3.0,5.0,3.0,5.0,3.0,1.0,1.0,1.0,4.0,1.0
4,1004,1.0,1.0,1.0,1.0,5.0,5.0,3.0,1.0,1.0,5.0,3.0,1.0


## 4. Handle Missing Values

In [14]:
# Check missing values in wide format
print('Missing values per question:')
print(wide_df.isnull().sum())

# Fill missing values with median of each question
question_cols = [col for col in wide_df.columns if col.startswith('Q')]
for col in question_cols:
    wide_df[col] = wide_df[col].fillna(wide_df[col].median())

# Verify no missing values remain
print('\nMissing values after filling:')
print(wide_df.isnull().sum())

Missing values per question:
Anonymized Student Number      0
Q2                           218
Q3                           104
Q4                           101
Q5                           272
Q6                           120
Q7                           139
Q8                           186
Q9                           119
Q10                          170
Q11                          299
Q12                           76
Q13                           81
dtype: int64

Missing values after filling:
Anonymized Student Number    0
Q2                           0
Q3                           0
Q4                           0
Q5                           0
Q6                           0
Q7                           0
Q8                           0
Q9                           0
Q10                          0
Q11                          0
Q12                          0
Q13                          0
dtype: int64


## 5. Scale Features

In [15]:
# Separate features and student IDs
student_ids = wide_df['Anonymized Student Number']
features = wide_df[question_cols]

# Scale features to [0, 1] range
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create final dataframe
final_df = pd.DataFrame(scaled_features, columns=question_cols)
final_df.insert(0, 'student_id', student_ids)

print('Final dataset shape:', final_df.shape)
print('\nSample of processed data:')
display(final_df.head())

Final dataset shape: (14697, 13)

Sample of processed data:


,student_id,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13
0,1000,1.0,0.00,0.0,1.0,0.5,0.5,0.5,0.0,0.0,0.0,0.50,0.5
1,1001,0.0,0.25,0.5,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.00,0.5
2,1002,0.0,0.25,0.5,1.0,1.0,0.5,0.5,0.0,0.0,0.5,0.00,0.0
3,1003,0.0,0.25,0.5,1.0,0.5,1.0,0.5,0.0,0.0,0.0,0.75,0.0
4,1004,0.0,0.00,0.0,0.0,1.0,1.0,0.5,0.0,0.0,1.0,0.50,0.0


## 6. Save Question Mapping for Reference

In [16]:
# Create a mapping of question numbers to full questions
question_mapping = pd.DataFrame({
    'question_id': [f'Q{i+1}' for i in range(len(questions))],
    'question_text': questions
})

print('Question mapping:')
display(question_mapping)

Question mapping:


,question_id,question_text
0,Q1,How often do you plan on doing your laundry?
1,Q2,How do you see yourself usually spending Frida...
2,Q3,How comfortable are you sharing belongings wit...
3,Q4,What kind of studying should happen in your room?
4,Q5,Which statement best describes your standards ...
5,Q6,How clean do you want your room to be?
6,Q7,How comfortable are you with any individual in...
7,Q8,How do you feel about groups of people sociali...
8,Q9,How would you describe your attitude toward sm...
9,Q10,How would you describe your attitude toward th...


## 7. Save Processed Data

In [17]:
# Save processed data and question mapping
final_df.to_csv('processed_data.csv', index=False)
question_mapping.to_csv('data/question_mapping.csv', index=False)

print('Files saved:')
print('1. data/processed_data.csv - Contains processed student responses')
print('2. data/question_mapping.csv - Contains mapping of question IDs to full questions')

Files saved:
1. data/processed_roommate_data.csv - Contains processed student responses
2. data/question_mapping.csv - Contains mapping of question IDs to full questions
